In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("../data/dados.csv", encoding="ISO-8859-1", sep=";", low_memory=False)
print(f'Linhas, Colunas: {df.shape} \n')
print(f'Informações: \n')
print(f'{df.info()}\n')


Linhas, Colunas: (224649, 385) 

Informações: 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 224649 entries, 0 to 224648
Columns: 385 entries, NU_ANO_CENSO to QT_TUR_ESP_CE
dtypes: float64(358), int64(11), object(16)
memory usage: 659.9+ MB
None



In [3]:
#Overview do dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 224649 entries, 0 to 224648
Columns: 385 entries, NU_ANO_CENSO to QT_TUR_ESP_CE
dtypes: float64(358), int64(11), object(16)
memory usage: 659.9+ MB


In [4]:
#Carrega o dataset de taxas de rendimento
df_tx = pd.read_excel("../data/tx_rend.xlsx", skiprows = 8, nrows = 129307)
print(f'Linhas, Colunas: {df_tx.shape} \n')
print(f'Informações: \n')
print(f'{df_tx.info()}\n')


Linhas, Colunas: (129307, 63) 

Informações: 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129307 entries, 0 to 129306
Data columns (total 63 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   NU_ANO_CENSO    129307 non-null  int64 
 1   NO_REGIAO       129307 non-null  object
 2   SG_UF           129307 non-null  object
 3   CO_MUNICIPIO    129307 non-null  int64 
 4   NO_MUNICIPIO    129307 non-null  object
 5   CO_ENTIDADE     129307 non-null  int64 
 6   NO_ENTIDADE     129307 non-null  object
 7   NO_CATEGORIA    129307 non-null  object
 8   NO_DEPENDENCIA  129307 non-null  object
 9   1_CAT_FUN       129307 non-null  object
 10  1_CAT_FUN_AI    129307 non-null  object
 11  1_CAT_FUN_AF    129307 non-null  object
 12  1_CAT_FUN_01    129307 non-null  object
 13  1_CAT_FUN_02    129307 non-null  object
 14  1_CAT_FUN_03    129307 non-null  object
 15  1_CAT_FUN_04    129307 non-null  object
 16  1_CAT_FUN_05    129307 non-

In [5]:
#Dados os datatypes criados a partir dos dois datasets:
df.info()
df_tx.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 224649 entries, 0 to 224648
Columns: 385 entries, NU_ANO_CENSO to QT_TUR_ESP_CE
dtypes: float64(358), int64(11), object(16)
memory usage: 659.9+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129307 entries, 0 to 129306
Data columns (total 63 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   NU_ANO_CENSO    129307 non-null  int64 
 1   NO_REGIAO       129307 non-null  object
 2   SG_UF           129307 non-null  object
 3   CO_MUNICIPIO    129307 non-null  int64 
 4   NO_MUNICIPIO    129307 non-null  object
 5   CO_ENTIDADE     129307 non-null  int64 
 6   NO_ENTIDADE     129307 non-null  object
 7   NO_CATEGORIA    129307 non-null  object
 8   NO_DEPENDENCIA  129307 non-null  object
 9   1_CAT_FUN       129307 non-null  object
 10  1_CAT_FUN_AI    129307 non-null  object
 11  1_CAT_FUN_AF    129307 non-null  object
 12  1_CAT_FUN_01    129307 non-null  object
 13  1_CAT_FUN_0

In [6]:
# Criar a lista de entidade para cada uma das bases
lista_co_entidades_microdados = pd.Series(df['CO_ENTIDADE'].to_list())
lista_co_entidades_tx_rend = pd.Series(df_tx['CO_ENTIDADE'].to_list())
print(f'O dataset de microdados possui {len(lista_co_entidades_microdados)} códigos de entidade, enquanto o dataset de taxa de rendimentos possui {len(lista_co_entidades_tx_rend)} códigos.\n')

#Obtendo os códigos presentes em um dataset mas ausente em outro
codigos_diferentes = [cod for cod in lista_co_entidades_microdados.to_list() if cod not in lista_co_entidades_tx_rend.to_list()]


O dataset de microdados possui 224649 códigos de entidade, enquanto o dataset de taxa de rendimentos possui 129307 códigos.



In [7]:
#Total de códigos diferentes
len(codigos_diferentes)

95342

In [8]:
#Criando um novo dataframe filtrando os microdados apenas para aqueles que não possuem CO_ENTIDADE que não 
#tem correspondente na base de taxa de rendimento
df_microdados = df.query('CO_ENTIDADE not in @codigos_diferentes').reset_index(drop=True)
df_microdados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129307 entries, 0 to 129306
Columns: 385 entries, NU_ANO_CENSO to QT_TUR_ESP_CE
dtypes: float64(358), int64(11), object(16)
memory usage: 379.8+ MB


In [9]:
#Seleciona as colunas relevantes a partir de seus índices
indices_col_tx = [5,45,57]
df_tx_filtrado = df_tx.iloc[:,indices_col_tx]

In [10]:
#Renomeia as colunas do Dataframe alvo para ficarem com valores mais claros
df_tx_filtrado = df_tx_filtrado.rename(columns={"3_CAT_FUN" : "ABAND_FUND", "3_CAT_MED" : "ABAND_MED"})
df_tx_filtrado

,CO_ENTIDADE,ABAND_FUND,ABAND_MED
0,11022558,0,--
1,11024291,0,--
2,11024372,0,--
3,11024666,0,--
4,11024682,0.7,8.4
...,...,...,...
129302,53084020,0,--
129303,53084039,0,--
129304,53084055,0,0
129305,53085000,0,--


In [11]:
#Filtra as colunas selecionadas apenas do dataset de microdados:
indices_col_microdados = ['IN_AGUA_POTAVEL', 'IN_ENERGIA_INEXISTENTE', 'IN_ESGOTO_INEXISTENTE', 'IN_BANHEIRO', 'IN_BIBLIOTECA_SALA_LEITURA', 'IN_COZINHA', 'IN_LABORATORIO_CIENCIAS', 'IN_LABORATORIO_INFORMATICA', 'IN_PARQUE_INFANTIL', 'IN_QUADRA_ESPORTES',
                         'IN_REFEITORIO', 'IN_LABORATORIO_EDUC_PROF', 'IN_SALA_MULTIUSO', 'IN_SALA_REPOUSO_ALUNO', 'IN_ACESSIBILIDADE_INEXISTENTE', 'QT_SALAS_UTILIZADAS', 'QT_SALAS_UTILIZA_CLIMATIZADAS', 'IN_DESKTOP_ALUNO', 'IN_COMP_PORTATIL_ALUNO', 'IN_TABLET_ALUNO', 'IN_INTERNET', 'IN_INTERNET_APRENDIZAGEM',
                         'IN_INTERNET_COMUNIDADE', 'IN_PROF_BIBLIOTECARIO', 'IN_PROF_SAUDE', 'IN_ALIMENTACAO', 
                          'IN_ESPACO_ATIVIDADE', 'IN_FUND', 'IN_MED']
df_microdados_filtrado = df_microdados.loc[:,indices_col_microdados]
df_microdados_filtrado

,IN_AGUA_POTAVEL,IN_ENERGIA_INEXISTENTE,IN_ESGOTO_INEXISTENTE,IN_BANHEIRO,IN_BIBLIOTECA_SALA_LEITURA,IN_COZINHA,IN_LABORATORIO_CIENCIAS,IN_LABORATORIO_INFORMATICA,IN_PARQUE_INFANTIL,IN_QUADRA_ESPORTES,...,IN_TABLET_ALUNO,IN_INTERNET,IN_INTERNET_APRENDIZAGEM,IN_INTERNET_COMUNIDADE,IN_PROF_BIBLIOTECARIO,IN_PROF_SAUDE,IN_ALIMENTACAO,IN_ESPACO_ATIVIDADE,IN_FUND,IN_MED
0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0
1,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
2,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0
3,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0
4,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,...,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129302,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
129303,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0
129304,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,...,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0
129305,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [12]:
# Salva os dataframes em arquivos, para um checkpoint
df_tx_filtrado.to_pickle("../data/tx_aband.pk1")
df_microdados_filtrado.to_feather("../data/microdados.feather")
df_tx_filtrado.info()
df_microdados_filtrado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129307 entries, 0 to 129306
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   CO_ENTIDADE  129307 non-null  int64 
 1   ABAND_FUND   129307 non-null  object
 2   ABAND_MED    129307 non-null  object
dtypes: int64(1), object(2)
memory usage: 3.0+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129307 entries, 0 to 129306
Data columns (total 29 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   IN_AGUA_POTAVEL                129307 non-null  float64
 1   IN_ENERGIA_INEXISTENTE         129307 non-null  float64
 2   IN_ESGOTO_INEXISTENTE          129307 non-null  float64
 3   IN_BANHEIRO                    129307 non-null  float64
 4   IN_BIBLIOTECA_SALA_LEITURA     129307 non-null  float64
 5   IN_COZINHA                     129307 non-null  float64
 6   IN_LABORATORIO_CIENCIAS   

In [3]:
#Carrega do backup
df_tx_use = pd.read_pickle("../data/tx_aband.pk1")
df_mc_use = pd.read_feather("../data/microdados.feather")

df_tx_use.info()
df_mc_use.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129307 entries, 0 to 129306
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   CO_ENTIDADE  129307 non-null  int64 
 1   ABAND_FUND   129307 non-null  object
 2   ABAND_MED    129307 non-null  object
dtypes: int64(1), object(2)
memory usage: 3.0+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129307 entries, 0 to 129306
Data columns (total 29 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   IN_AGUA_POTAVEL                129307 non-null  float64
 1   IN_ENERGIA_INEXISTENTE         129307 non-null  float64
 2   IN_ESGOTO_INEXISTENTE          129307 non-null  float64
 3   IN_BANHEIRO                    129307 non-null  float64
 4   IN_BIBLIOTECA_SALA_LEITURA     129307 non-null  float64
 5   IN_COZINHA                     129307 non-null  float64
 6   IN_LABORATORIO_CIENCIAS   

In [10]:
#Dividir as bases que tem fund e as que não tem IN_FUND e IN_MED
df_microdados_fund = df_microdados.query('IN_FUND == 1').reset_index(drop=True)
df_microdados_med = df_microdados.query('IN_MED == 1').reset_index(drop=True)

df_microdados_fund.info()
df_microdados_med.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120856 entries, 0 to 120855
Columns: 385 entries, NU_ANO_CENSO to QT_TUR_ESP_CE
dtypes: float64(358), int64(11), object(16)
memory usage: 355.0+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29384 entries, 0 to 29383
Columns: 385 entries, NU_ANO_CENSO to QT_TUR_ESP_CE
dtypes: float64(358), int64(11), object(16)
memory usage: 86.3+ MB


In [ ]:
codigos_diferentes = [cod for cod in lista_co_entidades.to_list() if cod not in lista_co_entidades_tx.to_list()]
codigos_diferentes

In [ ]:
set_of_co_ent = set(lista_co_entidades.to_list())
set_of_co_ent_tx = set(lista_co_entidades_tx.to_list())
print(f'len_co = {len(set_of_co_ent)}\n len_co_tx = {len(set_of_co_ent_tx)}')